# Plain TensorFlow (no Keras) on MNIST
Used the default Kaggle enviromnent. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


Load the data in a Pandas dataset and separetes the labels.

In [2]:
labeled_images = pd.read_csv('../input/train.csv')
test_images = pd.read_csv('../input/test.csv')

images=np.array(labeled_images.iloc[0:41500 , 1:])
labels = np.array(labeled_images.iloc[0:41500, :1]).ravel()

X_valid = np.array(labeled_images.iloc[41500: , 1:])
y_valid = np.array(labeled_images.iloc[41500:, :1]).ravel()

test_images= np.array(test_images.iloc[0:])





In [3]:
test_images


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Data Augmentation

Use this technique to increase the dataset. 
We could zoom in, out and tilt some degrees. But just shifted the images left, right, up and down by one pixel, creating four plus images from each dataset image. Now our dataset is 5X bigger.

This techique prevents overfitting and makes our model to generalize better. It's a welcome costless dataset increase. 

In [4]:
from scipy.ndimage.interpolation import shift
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

X_train_augmented = [image for image in images]
y_train_augmented = [label for label in labels]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):   #variação de 1 pixels em todas as direções
    for image, label in zip(images, labels):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train = np.array(X_train_augmented)
y_train = np.array(y_train_augmented)

X_train.shape

(207500, 784)

# Tensorflow
The google library for machine learning. 

At first, we define our input wich is 28X28 (image size in pixels)
Then we choose how many hidden layers we gonna use. In this case, the first hidden layer will have 300 nodes, and the second layer 100. This is an arbitrary decision, but definetly impacts the model performance. 2 hidden layers seems enough, but the number of nodes may. 300 and 100 seems to muuch, but the computation cost is not too big. 

The last layer (n_output) is the number of classes (0...9 digits)

lerning rate: too small and the network will never reach the optimum. Too big and the NN will diverge. 
batch normalization momentum : chose values close to 1

Dropout rate is the number of nodes that will be dropped during training phase. Dropout increases the NN performance and avoids overfitting.
This NN uses neurons with ELU activation function, because it works better than sigmoid and RELU. 

Also, some authors says that Nesterov is a better optimizer than ADAM


In [5]:
import tensorflow as tf
from functools import partial

tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

learning_rate=0.01
batch_norm_momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(), name='training')


dropout_rate =0.5   #dropout
X_drop= tf.layers.dropout(X,dropout_rate,training=training) #dropout

with tf.name_scope("dnn"):
    he_init = tf.variance_scaling_initializer()

    my_batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training=training,
            momentum=batch_norm_momentum)

    my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer=he_init)

    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)  #dropout
    bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
    hidden2 = my_dense_layer(bn1, n_hidden2, name="hidden2")
    bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training) #dropout
    logits_before_bn = my_dense_layer(bn2, n_outputs, name="outputs")
    logits = my_batch_norm_layer(logits_before_bn)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9, use_nesterov=True)  #usei Nesterov
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use tf.cast instead.


In [6]:
n_epochs = 50
batch_size = 200

In [7]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [8]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.984
1 Validation accuracy: 0.984
2 Validation accuracy: 0.984
3 Validation accuracy: 0.974
4 Validation accuracy: 0.984
5 Validation accuracy: 0.984
6 Validation accuracy: 0.988
7 Validation accuracy: 0.986
8 Validation accuracy: 0.986
9 Validation accuracy: 0.988
10 Validation accuracy: 0.988
11 Validation accuracy: 0.988
12 Validation accuracy: 0.986
13 Validation accuracy: 0.984
14 Validation accuracy: 0.984
15 Validation accuracy: 0.988
16 Validation accuracy: 0.992
17 Validation accuracy: 0.988
18 Validation accuracy: 0.986
19 Validation accuracy: 0.988
20 Validation accuracy: 0.988
21 Validation accuracy: 0.992
22 Validation accuracy: 0.986
23 Validation accuracy: 0.988
24 Validation accuracy: 0.99
25 Validation accuracy: 0.99
26 Validation accuracy: 0.992
27 Validation accuracy: 0.988
28 Validation accuracy: 0.986
29 Validation accuracy: 0.988
30 Validation accuracy: 0.99
31 Validation accuracy: 0.99
32 Validation accuracy: 0.992
33 Validation accuracy: 

In [9]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    X_scaled=test_images
    Z=logits.eval(feed_dict={X:X_scaled})
    results=np.argmax(Z,axis=1)
    
results

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


array([2, 0, 9, ..., 3, 9, 2])

# Kaggle results
If you are in the Kaggle contest, use the code below to submit your results.

In [10]:
submission = pd.DataFrame(results)
submission.index.name='ImageId'
submission.index+=1
submission.columns=['Label']
submission.to_csv('Submission.csv', header=True)

